Nama: Riski Abdi Rahmawan

NIM: 2241720060

Kelas: TI-3E

Absen: 25

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Praktikum 1 Bagging dengan RandomForest

*Import Library*

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

Kode tersebut mengimpor pustaka yang diperlukan untuk membangun model klasifikasi. Kode ini menggunakan **NumPy** dan **Pandas** untuk manipulasi data. Kemudian, **DecisionTreeClassifier** dan **RandomForestClassifier** dari **scikit-learn** diimpor untuk membuat model pohon keputusan dan hutan acak. Data dibagi menjadi set pelatihan dan pengujian menggunakan **train_test_split**. Selanjutnya, performa model diukur menggunakan **accuracy_score** dan **classification_report**, yang menampilkan akurasi dan laporan klasifikasi (presisi, recall, f1-score) dari prediksi model terhadap data uji.

*Persiapan data*

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/assets/wbc.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


Kode tersebut memuat dataset dari file CSV bernama **wbc.csv** menggunakan **Pandas** dan menyimpannya dalam variabel **df**. Fungsi **pd.read_csv()** digunakan untuk membaca file CSV. Setelah itu, **df.head()** menampilkan lima baris pertama dari dataset untuk memberikan gambaran awal tentang struktur dan isi data tersebut.

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


kode diatas untuk mengecek kolom yang bernilai null.

In [ ]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,3:-1]
y = df['diagnosis']
y = y.map({'M':1, 'B':0}) # Encode label

# Cek jumlah fitur dan instance
X.shape

(569, 29)

Kode tersebut melakukan seleksi fitur dan pemrosesan data untuk analisis. Pertama, variabel **X** diambil dari subset dataframe **df**, yaitu dari kolom ke-4 (indeks 3) hingga kolom kedua dari terakhir (tidak termasuk kolom terakhir). Variabel **y** diisi dengan nilai dari kolom **diagnosis**, yang kemudian di-encode menjadi angka: **M** (Malignant) menjadi **1**, dan **B** (Benign) menjadi **0**. Akhirnya, **X.shape** digunakan untuk mengecek dimensi (jumlah fitur dan instance) dari dataset yang telah dipilih, menampilkan jumlah baris (instance) dan kolom (fitur).

*Split data training dan testing*

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Kode tersebut membagi data menjadi set pelatihan dan set pengujian menggunakan fungsi **train_test_split** dari **scikit-learn**. Variabel **X** (fitur) dan **y** (label) dibagi secara acak, dengan **20%** data dialokasikan untuk pengujian (**test_size=0.2**) dan sisanya untuk pelatihan. Parameter **random_state=1** memastikan pembagian data yang konsisten setiap kali kode dijalankan. Hasil pembagian disimpan dalam empat variabel: **X_train** dan **X_test** untuk fitur pelatihan dan pengujian, serta **y_train** dan **y_test** untuk label pelatihan dan pengujian.

*Training Decision Tree*

In [ ]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.94
Test set accuracy: 0.9385964912280702


Kode tersebut membuat model klasifikasi menggunakan **DecisionTreeClassifier** dengan parameter default, yang menggunakan **Gini index** sebagai kriteria pemilihan fitur terbaik. Model dilatih dengan data pelatihan menggunakan metode **fit()**. Setelah pelatihan, model memprediksi label data uji dengan **predict()**, menghasilkan prediksi yang kemudian dibandingkan dengan label asli dari data uji. Akurasi model dihitung menggunakan **accuracy_score()**, yang mengukur persentase prediksi yang benar. Hasil akurasi ini ditampilkan untuk menunjukkan seberapa baik model memprediksi data uji.

*Training Random Forest*

In [ ]:
# Pada kasus kali ini kita akan menggunakan estimator pada RandomForest
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

rf = RandomForestClassifier(n_estimators=10, random_state=1)

# Sesuaikan dt ke set training
rf.fit(X_train, y_train)

# Memprediksi label set test
y_pred_rf = rf.predict(X_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 0.96
Test set accuracy: 0.956140350877193


Kode tersebut membuat model klasifikasi menggunakan **RandomForestClassifier** dengan parameter **n_estimators=10**, yang berarti model akan membangun 10 pohon keputusan untuk menghasilkan prediksi. Model dilatih menggunakan data pelatihan dengan metode **fit()**. Setelah pelatihan, model memprediksi label dari data uji menggunakan **predict()**, dan hasil prediksi dibandingkan dengan label asli dari data uji untuk menghitung akurasi model menggunakan **accuracy_score()**. Akurasi ini menunjukkan persentase prediksi yang benar dan ditampilkan dalam dua format output.

# Praktikum 2 Boosting dengan AdaBoost

*Import Library*

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder # Kebutuhan encoding label

Kode tersebut mengimpor pustaka yang diperlukan untuk membangun model klasifikasi. **NumPy** dan **Pandas** digunakan untuk manipulasi data, dengan **NumPy** menangani operasi numerik dan **Pandas** mengelola data berbasis tabel. Model klasifikasi yang digunakan meliputi **DecisionTreeClassifier** untuk membuat pohon keputusan, dan **AdaBoostClassifier**, yang menggabungkan beberapa model (seperti pohon keputusan) untuk meningkatkan akurasi prediksi melalui teknik boosting. **train_test_split** digunakan untuk membagi dataset menjadi set pelatihan dan pengujian. Evaluasi model dilakukan dengan **accuracy_score** dan **classification_report**, yang memberikan informasi tentang akurasi serta metrik lain seperti presisi, recall, dan f1-score. Selain itu, **LabelEncoder** digunakan untuk mengubah label kategorikal menjadi format numerik agar dapat digunakan dalam model pembelajaran mesin.

*Persiapan Data*

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/assets/iris.csv')

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Kode tersebut memuat dataset dari file CSV bernama **iris.csv** menggunakan **Pandas** dan menyimpannya dalam variabel **df**. File CSV tersebut dibaca menggunakan fungsi **pd.read_csv()**, yang mengimpor data ke dalam bentuk dataframe. Setelah itu, **df.head()** digunakan untuk menampilkan lima baris pertama dari dataset, memberikan gambaran tentang struktur dan isi data, seperti kolom dan beberapa contoh nilai dari dataset **Iris**.

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


Kode diatas digunakan untuk mengecek kolom yang bernilai null

In [ ]:
# Seleksi fitur
X = df.iloc[:,2:-1]
y = df['Species']

# encode label
ec = LabelEncoder()
y = ec.fit_transform(y)

# Cek jumlah fitur dan instance
print(X.shape)

# Cek label
print(y)

(150, 3)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Kode tersebut melakukan seleksi fitur dan pemrosesan label dari dataset **Iris**. Variabel **X** berisi fitur pengukuran dari kolom ke-3 hingga kolom kedua terakhir, sementara **y** berisi label spesies bunga dari kolom **Species**. Dengan menggunakan **LabelEncoder**, label spesies diubah menjadi format numerik. Kode ini kemudian mencetak dimensi dataset fitur dengan **X.shape** dan menampilkan hasil encoding label dengan **print(y)**.

*Split Data Training dan Test*

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Kode tersebut membagi dataset menjadi set pelatihan dan set pengujian menggunakan fungsi **train_test_split** dari **scikit-learn**. Data fitur **X** dan label **y** dibagi secara acak, dengan **20%** dari total data dialokasikan untuk pengujian (dengan parameter **test_size=0.2**), sedangkan sisanya untuk pelatihan. Parameter **random_state=1** memastikan bahwa pembagian data bersifat konsisten setiap kali kode dijalankan. Hasil pembagian ini disimpan dalam empat variabel: **X_train** dan **X_test** untuk fitur pelatihan dan pengujian, serta **y_train** dan **y_test** untuk label pelatihan dan pengujian.

*Training Decision Tree*

In [ ]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


Kode tersebut membangun model klasifikasi menggunakan **DecisionTreeClassifier** dengan parameter default, yang menggunakan **Gini index** sebagai kriteria pemilihan fitur. Model dilatih dengan data pelatihan menggunakan metode **fit()**. Setelah pelatihan, model memprediksi label dari data uji menggunakan **predict()**, dan hasil prediksi dibandingkan dengan label asli dari data uji. Akurasi model dihitung menggunakan **accuracy_score()**, yang mengukur persentase prediksi yang benar, dan hasil akurasi tersebut ditampilkan dalam dua format output, menunjukkan seberapa baik model pohon keputusan memprediksi data pengujian.

*Training AdaBoost*

In [ ]:
# Pada kasus kali ini kita akan menggunakan estimator pada AdaBoost
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

ada = AdaBoostClassifier(n_estimators=2)

# Sesuaikan dt ke set training
ada.fit(X_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(X_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Kode tersebut membangun model klasifikasi menggunakan **AdaBoostClassifier** dengan parameter **n_estimators=2**, yang berarti model akan menggunakan dua estimator (pohon keputusan) untuk meningkatkan akurasi prediksi. Model dilatih dengan data pelatihan melalui metode **fit()**. Setelah pelatihan, model memprediksi label dari data uji menggunakan **predict()**, dan hasil prediksi dibandingkan dengan label asli dari data uji. Akurasi model dihitung menggunakan **accuracy_score()**, yang menunjukkan persentase prediksi yang benar, dan hasil akurasi tersebut ditampilkan dalam dua format output, memberikan gambaran seberapa efektif model AdaBoost dalam memprediksi data pengujian.

# Praktikum 3 Stacking

*Stacking*

In [ ]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier



layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())


clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0.8947368421052632

Kode ini membangun model klasifikasi menggunakan **StackingClassifier** yang menggabungkan beberapa estimator dalam dua lapisan. Pertama, beberapa pustaka dari **scikit-learn** diimpor, termasuk **RandomForestClassifier**, **StackingClassifier**, **LogisticRegression**, **KNeighborsClassifier**, dan **DecisionTreeClassifier**. Estimator untuk lapisan pertama didefinisikan, terdiri dari **RandomForestClassifier** dengan 10 estimator dan **KNeighborsClassifier** dengan 5 tetangga terdekat. Selanjutnya, lapisan kedua berisi **DecisionTreeClassifier** dan **RandomForestClassifier** dengan 50 estimator. Model **layer_two** dibentuk menggunakan **StackingClassifier**, yang menggabungkan estimator dari lapisan kedua dengan **LogisticRegression** sebagai estimator akhir. Kemudian, dataset dibagi menjadi set pelatihan dan set pengujian menggunakan **train_test_split**, dengan stratifikasi berdasarkan label untuk menjaga proporsi kelas. Model dilatih menggunakan data pelatihan dengan metode **fit()**, dan akurasi model diukur menggunakan **score()** pada data pengujian. Secara keseluruhan, kode ini menciptakan dan melatih model stacking yang mengintegrasikan beberapa algoritma untuk meningkatkan kinerja klasifikasi.

# Praktikum 4 Voting

*Stacking dengan Voting*

*Import Library*

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB # import Naive Bayes model Gaussian (asumsi data terdistribusi normal)
from sklearn.svm import SVC # import SVM classifier
from sklearn.ensemble import VotingClassifier # import model Voting
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

Kode ini mengimpor pustaka yang diperlukan untuk membangun model klasifikasi menggunakan metode ensemble **VotingClassifier**. **NumPy** dan **Pandas** digunakan untuk manipulasi data, dengan **NumPy** menangani operasi numerik dan **Pandas** mengelola data dalam format tabel. Model **GaussianNB** diimpor untuk menerapkan Naive Bayes dengan asumsi bahwa data mengikuti distribusi normal, sementara **SVC** diimpor sebagai classifier Support Vector Machine (SVM), yang digunakan untuk klasifikasi dengan membangun hyperplane optimal di ruang fitur. **VotingClassifier** digunakan untuk menggabungkan prediksi dari beberapa model, memberikan hasil berdasarkan suara mayoritas atau rata-rata probabilitas, yang dapat meningkatkan akurasi klasifikasi. Selain itu, **train_test_split** digunakan untuk membagi dataset menjadi set pelatihan dan set pengujian, sedangkan **accuracy_score** dan **classification_report** digunakan untuk mengevaluasi performa model, dengan yang pertama mengukur akurasi prediksi dan yang kedua memberikan metrik tambahan seperti presisi, recall, dan f1-score. Secara keseluruhan, kode ini menyiapkan berbagai model dan metrik yang diperlukan untuk klasifikasi dan evaluasi model.

*Persiapan Data*

In [ ]:
# Load Data

dbt = pd.read_csv('/content/drive/MyDrive/Machine Learning/assets/diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Kode ini memuat dataset dari file CSV bernama **diabetes.csv** menggunakan **Pandas** dan menyimpannya dalam variabel **dbt**. Fungsi **pd.read_csv()** digunakan untuk membaca data dari file CSV dan mengimpor informasi ke dalam bentuk dataframe. Setelah itu, **dbt.head()** digunakan untuk menampilkan lima baris pertama dari dataset, memberikan gambaran awal tentang struktur dan isi data, termasuk kolom dan beberapa contoh nilai yang terdapat dalam dataset diabetes.

In [ ]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

Kode diatas digunakan untuk mengecek nama kolom yang terdapat pada diabetes.cvs

In [ ]:
# Cek kolom null
dbt.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


Kode diatas digunakan untuk mengecek kolom yang null.

In [ ]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


Kode ini memeriksa keberadaan nilai nol pada kolom-kolom tertentu dalam dataset diabetes untuk mengidentifikasi fitur yang mungkin tidak realistis jika memiliki nilai nol. Dalam konteks medis, parameter seperti **Glucose**, **BloodPressure**, dan **Insulin** seharusnya memiliki nilai lebih dari nol pada individu yang sehat. Kode ini menginisialisasi daftar kolom yang akan diperiksa, yaitu **Pregnancies**, **Glucose**, **BloodPressure**, **SkinThickness**, **Insulin**, **BMI**, **DiabetesPedigreeFunction**, dan **Age**.

Kemudian, sebuah loop digunakan untuk iterasi melalui setiap kolom dalam **feature_columns**. Dalam setiap iterasi, kode mencetak jumlah nilai nol yang ditemukan dalam kolom tersebut dengan menggunakan **len(dbt.loc[dbt[column] == 0])**. Dengan cara ini, pengguna dapat melihat berapa banyak nilai nol yang ada di setiap kolom, yang kemudian dapat ditangani melalui proses imputasi untuk menggantikan nilai-nilai nol tersebut dengan nilai sintetis, seperti nilai rata-rata, agar dataset menjadi lebih realistis dan sesuai dengan konteks medis.

In [ ]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

Kode ini melakukan imputasi nilai nol pada kolom-kolom tertentu dalam dataset diabetes dengan menggantinya menggunakan nilai rata-rata (mean) dari kolom tersebut. Untuk melakukan ini, **SimpleImputer** dari **scikit-learn** diimpor dengan parameter **missing_values=0** untuk menunjukkan bahwa nilai yang hilang yang ingin diimputasi adalah nol. Strategi yang digunakan untuk imputasi ditentukan sebagai **"mean"**, yang berarti nilai nol akan diganti dengan rata-rata dari masing-masing kolom.

Dengan **copy=False**, data asli dalam **dbt** akan dimodifikasi secara langsung tanpa membuat salinan. Setelah itu, **fit_transform()** digunakan untuk menghitung nilai rata-rata dan menerapkan imputasi pada kolom yang terdaftar dalam **feature_columns**. Dengan cara ini, semua nilai nol dalam kolom tersebut akan digantikan dengan nilai rata-rata yang sesuai, sehingga meningkatkan kualitas dataset untuk analisis lebih lanjut dan pemodelan.

*Split Data Training dan Testing*

In [ ]:
X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Kode ini mempersiapkan dataset untuk pelatihan dan pengujian model klasifikasi. Pertama, variabel **X** diisi dengan fitur-fitur yang relevan dari dataframe **dbt**, yaitu kolom-kolom yang ada dalam **feature_columns**. Variabel **y** diisi dengan label target, yaitu kolom **Outcome** dari dataset, yang menunjukkan apakah seseorang terdiagnosis diabetes (biasanya dalam bentuk 0 atau 1).

Selanjutnya, dataset dibagi menjadi set pelatihan dan set pengujian menggunakan fungsi **train_test_split**. Dalam hal ini, 30% dari data akan digunakan sebagai set pengujian (**test_size=0.3**), sementara 70% akan digunakan untuk pelatihan. Parameter **random_state=42** ditetapkan untuk memastikan bahwa pembagian dataset dapat direproduksi, sehingga hasilnya konsisten setiap kali kode dijalankan. Setelah eksekusi kode ini, variabel **X_train** dan **y_train** akan berisi data pelatihan, sedangkan **X_test** dan **y_test** akan berisi data pengujian, siap untuk digunakan dalam pelatihan dan evaluasi model.

*Training dengan GaussianNB*

*Standarisasi Fitur*

In [ ]:
# Karena asumsi Gaussian NB adalah data terdistribusi secara normal,
# maka kita perlu melakukan standarisasi

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

Kode ini melakukan standarisasi pada fitur dalam dataset pelatihan dan pengujian agar memenuhi asumsi distribusi normal yang diperlukan oleh Gaussian Naive Bayes. **StandardScaler** dari **scikit-learn** diimpor dan digunakan untuk menghitung rata-rata dan deviasi standar dari data pelatihan (**X_train**) dengan metode **fit_transform()**, kemudian hasilnya disimpan dalam **X_train_std**. Selanjutnya, **transform()** diterapkan pada data pengujian (**X_test**) untuk menstandarisasi menggunakan parameter yang sama dari data pelatihan, dan hasilnya disimpan dalam **X_test_std**. Dengan cara ini, kedua set data memiliki rata-rata 0 dan deviasi standar 1.

*Training dan Evaluasi*

In [ ]:
# Buat obyek GaussianNB
gnb_std = GaussianNB()

# Fit dengan data yang telah di standarisasi
gnb_std.fit(X_train_std, y_train)

# Prediksi dengan data test
y_pred_gnb = gnb_std.predict(X_test_std)

# Evaluasi akurasi testing data
acc_gnb = accuracy_score(y_test, y_pred_gnb)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_gnb))
print(f"Test set accuracy: {acc_gnb}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


Kode ini membuat dan melatih model klasifikasi menggunakan Gaussian Naive Bayes pada data yang telah distandarisasi. Pertama, objek **GaussianNB** diinisialisasi dan disimpan dalam variabel **gnb_std**. Kemudian, metode **fit()** digunakan untuk melatih model dengan data pelatihan yang telah distandarisasi (**X_train_std**) dan label target (**y_train**).

Setelah model dilatih, prediksi dilakukan pada data pengujian yang juga telah distandarisasi (**X_test_std**) menggunakan metode **predict()**, dan hasil prediksi disimpan dalam variabel **y_pred_gnb**. Untuk mengevaluasi kinerja model, **accuracy_score** digunakan untuk menghitung akurasi dengan membandingkan label asli (**y_test**) dan prediksi yang dihasilkan. Hasil akurasi dicetak ke layar dengan format yang jelas, menunjukkan seberapa baik model Gaussian Naive Bayes melakukan klasifikasi pada data pengujian.

*Training dengan SVM Linier*

In [ ]:
# Model SVM linier tanpa tunnning hyperparameter
svm_lin = SVC(kernel='linear')

# Fit ke model
svm_lin.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_lin = svm_lin.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_lin = accuracy_score(y_test, y_pred_svm_lin)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_lin))
print(f"Test set accuracy: {acc_svm_lin}")

Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


Kode ini membuat dan melatih model klasifikasi menggunakan Support Vector Machine (SVM) dengan kernel linear tanpa melakukan penyesuaian hyperparameter. Pertama, objek **SVC** dari **scikit-learn** diinisialisasi dengan parameter **kernel='linear'** dan disimpan dalam variabel **svm_lin**. Kemudian, metode **fit()** digunakan untuk melatih model dengan data pelatihan yang telah distandarisasi (**X_train_std**) dan label target (**y_train**).

Setelah model dilatih, prediksi dilakukan pada data pengujian yang telah distandarisasi (**X_test_std**) menggunakan metode **predict()**, dan hasil prediksi disimpan dalam variabel **y_pred_svm_lin**. Untuk mengevaluasi kinerja model, **accuracy_score** digunakan untuk menghitung akurasi dengan membandingkan label asli (**y_test**) dengan prediksi yang dihasilkan. Hasil akurasi dicetak ke layar dengan format yang jelas, menunjukkan seberapa baik model SVM dengan kernel linear dalam melakukan klasifikasi pada data pengujian.

*Training dengan SVM RBF*

In [ ]:
# Model SVM RBF tanpa tunnning hyperparameter
svm_rbf = SVC(kernel='rbf')

# Fit ke model
svm_rbf.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_rbf = svm_rbf.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_rbf = accuracy_score(y_test, y_pred_svm_rbf)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_rbf))
print(f"Test set accuracy: {acc_svm_rbf}")

Test set accuracy: 0.72
Test set accuracy: 0.7229437229437229


Kode ini membuat dan melatih model klasifikasi menggunakan Support Vector Machine (SVM) dengan kernel Radial Basis Function (RBF) tanpa melakukan penyesuaian hyperparameter. Pertama, objek **SVC** dari **scikit-learn** diinisialisasi dengan parameter **kernel='rbf'** dan disimpan dalam variabel **svm_rbf**. Selanjutnya, model dilatih dengan data pelatihan yang telah distandarisasi (**X_train_std**) dan label target (**y_train**) menggunakan metode **fit()**.

Setelah proses pelatihan, prediksi dilakukan pada data pengujian yang juga telah distandarisasi (**X_test_std**) dengan metode **predict()**, dan hasilnya disimpan dalam variabel **y_pred_svm_rbf**. Untuk mengevaluasi kinerja model, **accuracy_score** digunakan untuk menghitung akurasi dengan membandingkan label asli (**y_test**) dengan prediksi yang dihasilkan. Hasil akurasi dicetak ke layar dengan format yang jelas, menunjukkan efektivitas model SVM dengan kernel RBF dalam melakukan klasifikasi pada data pengujian.

*Training dengan Voting*

In [ ]:
# Definisikan algoritma yang akan digunakan untuk voting

clf1 = GaussianNB()
clf2 = SVC(kernel='linear')
clf3 = SVC(kernel='rbf', probability=True)

# model hard voting
voting = VotingClassifier(estimators=[('GaussianNB', clf1), ('SVM-LIN', clf2), ('SVM-RBF', clf3)], voting='hard')

# Fit model
voting.fit(X_train_std, y_train)

# Prediksi
y_pred_vt1 = voting.predict(X_test_std)

# Evaluasi akurasi testing data
acc_vt1 = accuracy_score(y_test, y_pred_vt1)

# Print hasil evaluasi
print('Voting Hard')
print("Test set accuracy: {:.2f}".format(acc_vt1))
print(f"Test set accuracy: {acc_vt1}")

Voting Hard
Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


Kode ini mendefinisikan dan melatih model klasifikasi menggunakan metode *hard voting*, yang menggabungkan prediksi dari beberapa algoritma untuk meningkatkan akurasi. Pertama, tiga algoritma klasifikasi diinisialisasi: **GaussianNB** untuk Naive Bayes, **SVC** dengan kernel linear, dan **SVC** dengan kernel Radial Basis Function (RBF) yang diatur untuk menghasilkan probabilitas prediksi.

Model *hard voting* dibangun menggunakan **VotingClassifier** dengan ketiga algoritma yang disebutkan. Metode ini menggabungkan prediksi dari semua model, di mana kelas dengan suara terbanyak (prediksi terbanyak) akan dipilih sebagai hasil akhir.

Setelah model dibangun, metode **fit()** digunakan untuk melatih model dengan data pelatihan yang telah distandarisasi (**X_train_std**) dan label target (**y_train**). Selanjutnya, prediksi dilakukan pada data pengujian yang juga telah distandarisasi (**X_test_std**) menggunakan metode **predict()**, dan hasilnya disimpan dalam variabel **y_pred_vt1**. Untuk mengevaluasi kinerja model, **accuracy_score** menghitung akurasi dengan membandingkan label asli (**y_test**) dengan prediksi yang dihasilkan. Hasil akurasi dicetak ke layar dengan format yang jelas, menunjukkan efektivitas metode *hard voting* dalam melakukan klasifikasi pada data pengujian.

#Tugas Praktikum

***Tugas 1***

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/assets/mushrooms.csv')

# Preprocess data
le = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

# Split data
X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], test_size=0.2, random_state=42)

# Define hyperparameter tuning space
param_grid_dt = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Perform hyperparameter tuning
grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5, scoring='accuracy')
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='accuracy')

grid_dt.fit(X_train, y_train)
grid_rf.fit(X_train, y_train)

# Evaluate models
y_pred_dt = grid_dt.predict(X_test)
y_pred_rf = grid_rf.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt))

print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

Decision Tree Accuracy: 1.0
Random Forest Accuracy: 1.0
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



Pertama, kode tersebut memuat library yang diperlukan, yaitu pandas untuk pengolahan data, sklearn untuk algoritma machine learning, dan LabelEncoder untuk encoding data kategorik. Kemudian, kode tersebut memuat dataset mushroom dari file csv dan melakukan preprocessing data dengan menggunakan LabelEncoder untuk mengubah data kategorik menjadi numerik. Setelah itu, kode tersebut membagi data menjadi dua bagian, yaitu data latih (X_train dan y_train) dan data uji (X_test dan y_test) dengan menggunakan fungsi train_test_split dari sklearn. Selanjutnya, kode tersebut mendefinisikan ruang hyperparameter untuk algoritma Decision Tree dan Random Forest, yang digunakan untuk melakukan tuning hyperparameter dengan menggunakan GridSearchCV dari sklearn. Kode tersebut kemudian melakukan tuning hyperparameter untuk algoritma Decision Tree dan Random Forest dengan menggunakan GridSearchCV, yang melakukan pencarian secara sistematis terhadap kombinasi hyperparameter yang optimal untuk meningkatkan akurasi model. Setelah melakukan tuning hyperparameter, kode tersebut melakukan evaluasi model dengan menggunakan data uji, yang dilakukan dengan menggunakan fungsi accuracy_score, classification_report, dan confusion_matrix dari sklearn. Terakhir, kode tersebut mencetak hasil evaluasi model, yaitu akurasi, laporan klasifikasi, dan matriks kekeliruan untuk algoritma Decision Tree dan Random Forest, sehingga dapat membantu dalam memilih algoritma yang paling baik untuk klasifikasi data mushroom dan meningkatkan akurasi model dengan melakukan tuning hyperparameter.

***Tugas 2***

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/assets/mushrooms.csv')

# Preprocess data
le = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

# Split data
X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], test_size=0.2, random_state=42)

# Define hyperparameter tuning space
param_grid_dt = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

param_grid_ab = {
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.1, 1, 10]
}

# Perform hyperparameter tuning
grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5, scoring='accuracy')
grid_ab = GridSearchCV(AdaBoostClassifier(), param_grid_ab, cv=5, scoring='accuracy')

grid_dt.fit(X_train, y_train)
grid_ab.fit(X_train, y_train)

# Evaluate models
y_pred_dt = grid_dt.predict(X_test)
y_pred_ab = grid_ab.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ab))

print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt))

print("AdaBoost Classification Report:")
print(classification_report(y_test, y_pred_ab))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

Decision Tree Accuracy: 1.0
AdaBoost Accuracy: 1.0
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625

AdaBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



Pertama, kode tersebut memuat library yang diperlukan, yaitu pandas untuk pengolahan data, sklearn untuk algoritma machine learning, dan LabelEncoder untuk encoding data kategorik. Kemudian, kode tersebut memuat dataset mushroom dari file csv dan melakukan preprocessing data dengan menggunakan LabelEncoder untuk mengubah data kategorik menjadi numerik. Setelah itu, kode tersebut membagi data menjadi dua bagian, yaitu data latih (X_train dan y_train) dan data uji (X_test dan y_test) dengan menggunakan fungsi train_test_split dari sklearn. Selanjutnya, kode tersebut mendefinisikan ruang hyperparameter untuk algoritma Decision Tree dan AdaBoost, yang digunakan untuk melakukan tuning hyperparameter dengan menggunakan GridSearchCV dari sklearn. Kode tersebut kemudian melakukan tuning hyperparameter untuk algoritma Decision Tree dan AdaBoost dengan menggunakan GridSearchCV, yang melakukan pencarian secara sistematis terhadap kombinasi hyperparameter yang optimal untuk meningkatkan akurasi model. Setelah melakukan tuning hyperparameter, kode tersebut melakukan evaluasi model dengan menggunakan data uji, yang dilakukan dengan menggunakan fungsi accuracy_score, classification_report, dan confusion_matrix dari sklearn. Terakhir, kode tersebut mencetak hasil evaluasi model, yaitu akurasi, laporan klasifikasi, dan matriks kekeliruan untuk algoritma Decision Tree dan AdaBoost, sehingga dapat membantu dalam memilih algoritma yang paling baik untuk klasifikasi data mushroom dan meningkatkan akurasi model dengan melakukan tuning hyperparameter.

***Tugas 3***

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load dataset diabetes
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/assets/diabetes.csv')

# Split data
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definisikan algoritma yang akan digunakan dalam ensemble
lr = LogisticRegression()
svm = SVC(kernel='poly', degree=2)
dt = DecisionTreeClassifier()

# Buat ensemble voting
ensemble = VotingClassifier(estimators=[('lr', lr), ('svm', svm), ('dt', dt)])

# Train ensemble
ensemble.fit(X_train, y_train)

# Prediksi
y_pred = ensemble.predict(X_test)

# Evaluasi
print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))
print("Matriks Kekeliruan:")
print(confusion_matrix(y_test, y_pred))

Akurasi: 0.7662337662337663
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.81      0.83      0.82        99
           1       0.68      0.65      0.67        55

    accuracy                           0.77       154
   macro avg       0.75      0.74      0.74       154
weighted avg       0.76      0.77      0.77       154

Matriks Kekeliruan:
[[82 17]
 [19 36]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pertama, kode tersebut memuat library yang diperlukan, yaitu pandas untuk pengolahan data dan sklearn untuk algoritma machine learning. Kemudian, kode tersebut memuat dataset diabetes dari file csv dan membagi data menjadi dua bagian, yaitu data latih (X_train dan y_train) dan data uji (X_test dan y_test) dengan menggunakan fungsi train_test_split dari sklearn. Selanjutnya, kode tersebut mendefinisikan algoritma yang akan digunakan dalam ensemble, yaitu Logistic Regression, SVM kernel polynomial, dan Decision Tree. Kode tersebut kemudian membuat ensemble voting dengan menggunakan VotingClassifier dari sklearn dan melakukan training ensemble dengan menggunakan data latih. Setelah itu, kode tersebut melakukan prediksi dengan menggunakan data uji dan melakukan evaluasi dengan menggunakan fungsi accuracy_score, classification_report, dan confusion_matrix dari sklearn. Terakhir, kode tersebut mencetak hasil evaluasi, yaitu akurasi, laporan klasifikasi, dan matriks kekeliruan untuk ensemble voting, sehingga dapat membantu dalam memilih algoritma yang paling baik untuk klasifikasi data diabetes dan meningkatkan akurasi model dengan melakukan ensemble voting.